
Created on Fri Sep 18 14:29 2021 (Author: Clara Burgard)

This is an example script to apply multimelt



In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import trange, tqdm
import time
import multimelt.melt_functions as meltf
import multimelt.useful_functions as uf
from multimelt.constants import *
import os, glob

READ IN DATA

In [ ]:
inputpath_data='/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/NEMO_eORCA025.L121_OPM026_ANT_STEREO/'
inputpath_profiles = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/T_S_PROF/nemo_5km_OPM026/'
inputpath_mask = '/bettik/burgardc/DATA/TEST_MULTIMELT/'
inputpath_boxes = '/bettik/burgardc/DATA/TEST_MULTIMELT/'
inputpath_plumes = '/bettik/burgardc/DATA/TEST_MULTIMELT/'
inputpath_deepmelt = '/bettik/burgardc/DATA/TEST_MULTIMELT/'

outputpath_melt = '/bettik/burgardc/DATA/TEST_MULTIMELT/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc') #file containing 'corrected_isfdraft'
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim)
file_conc = xr.open_dataset(inputpath_data+'isfdraft_conc_Ant_stereo.nc') #file containing 'isfdraft_conc'
file_conc_cut = uf.cut_domain_stereo(file_conc, map_lim, map_lim)

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'mask_file.nc') #file containing 'ISF_mask','latitude', 'longitude','front_bot_depth_avg', 'front_bot_depth_max','isf_name'
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)

file_TS_orig = xr.open_dataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018.nc') #file containing 'theta_ocean' and 'salinity_ocean'
file_TS = file_TS_orig.sel(Nisf=large_isf)

In [ ]:
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'boxes_2D.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'boxes_1D.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'plume_characteristics.nc')
plume_charac = plume_charac.assign_coords(
    option=['lazero' if v == 'new_lazero' else v for v in plume_charac.option.values])
deepmelt_charac = xr.open_dataset(inputpath_deepmelt+'deepmelt_slope_info_bedrock_draft_latlon.nc')

PREPARE DATA

In [ ]:
lon = file_isf.longitude
lat = file_isf.latitude

In [ ]:
file_isf_conc = file_conc_cut['isfdraft_conc']

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos

In [ ]:
isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL', 'dIF']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name','front_ice_depth_min']]

In [ ]:
geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos).rename({'corrected_isfdraft':'ice_draft_pos'}).merge(grid_cell_area_weighted).rename({'isfdraft_conc':'grid_cell_area_weighted'}).merge(file_isf_conc)
geometry_info_1D = param_var_of_int_1D

SIMPLE PARAMS

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

mparam = 'quadratic_local_locslope'# POSSIBILITIES: ['linear_local', 'quadratic_local', 'quadratic_local_locslope', 'quadratic_local_cavslope', 'quadratic_mixed_mean', 'quadratic_mixed_locslope','quadratic_mixed_cavslope'] 

gamma = 7.1e-5 # fill in
ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, 
                                                           T_S_profile, 
                                                           geometry_info_2D, 
                                                           geometry_info_1D, 
                                                           isf_stack_mask, 
                                                           mparam, 
                                                           gamma, 
                                                           U_param=True)


ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'.nc')
ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'.nc')

PLUMES

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

mparam = 'lazero19'# POSSIBILITIES: ['lazero19', 'lazero19_modif']

gamma = 9.3e-4# fill in
E0 = 3.0e-2# fill in
                 
ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, 
                                                           T_S_profile, 
                                                           geometry_info_2D, 
                                                           geometry_info_1D, 
                                                           isf_stack_mask, 
                                                           mparam, 
                                                           gamma, 
                                                           E0=E0, 
                                                           verbose=True)
            
ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'.nc')
ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'.nc')

BOXES

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 
picop_opt = 'no'

nD_config = 4# POSSIBILITIES: 1 to 4
pism_version = 'no'# POSSIBILITIES: 'yes' or 'no'

mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picop'+picop_opt

C = 2.9e6# fill in
gamma = 0.88e-5# fill in

ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, 
                                                           T_S_profile, 
                                                           geometry_info_2D, 
                                                           geometry_info_1D, 
                                                           isf_stack_mask, 
                                                           mparam, 
                                                           gamma,
                                                           C=C, 
                                                           angle_option='local', 
                                                           box_charac_2D=box_charac_all_2D, 
                                                           box_charac_1D=box_charac_all_1D, 
                                                           box_tot=nD_config, 
                                                           box_tot_option='nD_config', 
                                                           pism_version=pism_version, 
                                                           picop_opt=picop_opt)
                                       
ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'.nc')
ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'.nc')

PICOP

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 

nD_config = 4# POSSIBILITIES: 1 to 4    
pism_version = 'yes'# POSSIBILITIES: 'yes' or 'no'
picop_opt = '2019'# POSSIBILITIES: '2018' or '2019'

mparam = 'boxes_'+str(nD_config)+'_pism'+pism_version+'_picopyes'

C = 2.9e6# for box part - fill in
gamma = 0.88e-5# for box part - fill in

gamma_plume = 9.3e-4# for plume part - fill in
E0 = 3.0e-2# for plume part - fill in

ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, 
                                                           T_S_profile, 
                                                           geometry_info_2D, 
                                                           geometry_info_1D, 
                                                           isf_stack_mask, 
                                                           mparam, 
                                                           gamma,
                                                           C=C,
                                                           E0=E0, 
                                                           angle_option='local',
                                                           box_charac_2D=box_charac_all_2D, 
                                                           box_charac_1D=box_charac_all_1D, 
                                                           box_tot=nD_config, 
                                                           box_tot_option='nD_config', 
                                                           pism_version=pism_version,
                                                           picop_opt=picop_opt, 
                                                           gamma_plume=gamma_plume)

ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'.nc')
ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'.nc')

DEEPMELT

In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth') 

mparam = 'DeepMelt'

path_model = '/bettik/burgardc/SCRIPTS/multimelt/deepmelt_extras/'

geometry_info_2D_extended = geometry_info_2D.merge(file_isf[['dGL','dIF']]).merge(deepmelt_charac)

deepmelt_norm_file = xr.open_dataset(path_model+'DeepMelt_norm_metrics.nc')
deepmelt_norm = deepmelt_norm_file.to_dataframe()

ds_2D, ds_1D = meltf.calculate_melt_rate_1D_and_2D_all_isf(nisf_list, 
                                                           T_S_profile, 
                                                           geometry_info_2D_extended, 
                                                           geometry_info_1D, 
                                                           isf_stack_mask, 
                                                           mparam, 
                                                           None,
                                                           deepmelt_model=path_model+'DeepMelt_',
                                                           deepmelt_norm=deepmelt_norm)

ds_2D.to_netcdf(outputpath_melt+'melt_rates_2D_'+mparam+'.nc')
ds_1D.to_netcdf(outputpath_melt+'melt_rates_1D_'+mparam+'.nc')